In [56]:
import pandas as pd
import re
from collections import *
pd.options.mode.chained_assignment = None  # default='warn'

In [11]:
uspto_ = pd.read_csv('names.csv')

In [19]:
uspto_ = uspto_.name.drop_duplicates().reset_index()

In [110]:
uspto_.to_csv('uspto.csv',index=False)

In [114]:
from fuzzywuzzy import fuzz

In [115]:
fuzz.token_set_ratio('a intl', 'a international')

57

In [111]:
funda = pd.read_csv('../funda50-17.csv',low_memory=False,usecols=['gvkey','cik','conm'])

In [107]:
cik = pd.read_csv('coname.csv')
cik.columns=['cik','date','name','fiscal','industry','indcode']
cik = cik[['cik','name']]
cik_name = cik.drop_duplicates(subset='name')
cik_name['name'] = cik_name.name.str.replace('/.{2,3}/{0,1}','').str.strip('/').str.strip()
comp_name = funda[['gvkey','cik','conm']].drop_duplicates()
comp_name['conm'] = comp_name.conm.str.replace('-CL [A-D]','').str.replace('-old','',re.I).str.strip()
allnames = comp_name.merge(cik_name,on='cik',how='left')
allnames['name'] = allnames['name'].fillna(allnames.conm)
allnames = allnames.drop_duplicates(subset='name')
allnames= allnames[['gvkey','name']]
allnames.to_csv('base_name.csv',index=False)

In [89]:
name_set = dict()
for gvkey, name in allnames.values:
    x = re.split('\s+',name)
    if gvkey in name_set:
        for x in name:
            name_set[gvkey].add(x.lower())
    else:
        name_set[gvkey] = set([x.lower() for x in x])

word_list = []
for v in name_set.values():
    word_list.extend(list(v))

ct = Counter(word_list)

unique_word = [word for word,n in ct.most_common() if n==1]   

In [4]:
coname__ = pd.read_csv('__coname__.csv',names=['id','name','gvkey','comp_name','type'])

In [5]:
uspto_merged = uspto_.merge(coname__,on='name',how='left')

In [35]:
uspto_merged.to_csv('uspto_merged.csv',index=False)